In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test3/test_最寄駅_面積（㎡）_容積率（％）.csv'
data_cleaned = pd.read_csv(data_path)

print(data_cleaned.head())

        ID  市区町村コード  最寄駅：距離（分）   間取り  面積（㎡）     建築年 建物の構造  建ぺい率（％）  容積率（％）  \
0  1000000     1101       26.0  ３ＬＤＫ   75.0  1989.0    ＲＣ     40.0    60.0   
1  1000056     1101        1.0  ２ＬＤＫ   55.0  2016.0    ＲＣ     80.0   600.0   
2  1000108     1101        2.0    １Ｒ   15.0  1989.0   ＳＲＣ     80.0   400.0   
3  1000109     1101        2.0  １ＬＤＫ   45.0  1991.0   ＳＲＣ     80.0   400.0   
4  1000110     1101        3.0    １Ｒ   20.0  1981.0    ＲＣ     80.0   400.0   

     取引時点  
0  202006  
1  202003  
2  202006  
3  202006  
4  202006  


1. 新しい特徴量エンジニアリングを作成する

In [3]:
import pandas as pd
import numpy as np

# データの読み込み
data_path = '/Users/hayakawakazue/Downloads/data/test3/test_最寄駅_面積（㎡）_容積率（％）.csv'
test_data_cleaned = pd.read_csv(data_path)

# ダミーの取引価格（総額）_log列を追加
test_data_cleaned['取引価格（総額）_log'] = 0  # ダミー値として0を使用

# 価格/面積比を計算
test_data_cleaned['価格/面積比'] = test_data_cleaned['取引価格（総額）_log'] / test_data_cleaned['面積（㎡）']

# 取引時点から年を抽出
test_data_cleaned['取引年'] = test_data_cleaned['取引時点'].astype(str).str[:4].astype(int)

# 築年数を計算（取引年 - 建築年）
test_data_cleaned['築年数'] = test_data_cleaned['取引年'] - test_data_cleaned['建築年']

# 取引月を抽出
test_data_cleaned['取引月'] = test_data_cleaned['取引時点'].apply(lambda x: int(str(x)[4:6]))

# 面積/築年数比を計算
test_data_cleaned['面積/築年数比'] = test_data_cleaned['面積（㎡）'] / test_data_cleaned['築年数']

# エリア人気度スコアの計算
area_popularity = test_data_cleaned.groupby('市区町村コード')['取引価格（総額）_log'].mean().reset_index()
area_popularity.columns = ['市区町村コード', 'エリア人気度スコア']
test_data_cleaned = test_data_cleaned.merge(area_popularity, on='市区町村コード', how='left')

# 年別価格トレンドの計算
yearly_trend = test_data_cleaned.groupby('取引年')['取引価格（総額）_log'].mean().reset_index()
yearly_trend.columns = ['取引年', '年別価格トレンド']
test_data_cleaned = test_data_cleaned.merge(yearly_trend, on='取引年', how='left')

# 価格増加率の計算
test_data_cleaned['価格増加率'] = test_data_cleaned['取引価格（総額）_log'].pct_change().fillna(0)

# 駅距離/面積の計算
test_data_cleaned['駅距離_面積比'] = test_data_cleaned['最寄駅：距離（分）'] / test_data_cleaned['面積（㎡）']

# 容積率と建ぺい率の比率
test_data_cleaned['容積率_建ぺい率比'] = test_data_cleaned['容積率（％）'] / test_data_cleaned['建ぺい率（％）']

# 面積と容積率の交互作用
test_data_cleaned['面積×容積率'] = test_data_cleaned['面積（㎡）'] * test_data_cleaned['容積率（％）']

# 建築年と建ぺい率の交互作用
test_data_cleaned['建築年×建ぺい率'] = test_data_cleaned['建築年'] * test_data_cleaned['建ぺい率（％）']

# 新しい特徴量を含むデータの確認
print(test_data_cleaned.head())

# データを保存
test_data_cleaned_path = '/Users/hayakawakazue/Downloads/data/test4/test_cleaned_feature_engineered.csv'
test_data_cleaned.to_csv(test_data_cleaned_path, index=False)

print(f"データを保存しました: {test_data_cleaned_path}")

# データの行数と列数をプリントする
print(f"\nデータの行数: {test_data_cleaned.shape[0]}")
print(f"データの列数: {test_data_cleaned.shape[1]}")


        ID  市区町村コード  最寄駅：距離（分）   間取り  面積（㎡）     建築年 建物の構造  建ぺい率（％）  容積率（％）  \
0  1000000     1101       26.0  ３ＬＤＫ   75.0  1989.0    ＲＣ     40.0    60.0   
1  1000056     1101        1.0  ２ＬＤＫ   55.0  2016.0    ＲＣ     80.0   600.0   
2  1000108     1101        2.0    １Ｒ   15.0  1989.0   ＳＲＣ     80.0   400.0   
3  1000109     1101        2.0  １ＬＤＫ   45.0  1991.0   ＳＲＣ     80.0   400.0   
4  1000110     1101        3.0    １Ｒ   20.0  1981.0    ＲＣ     80.0   400.0   

     取引時点  ...   築年数  取引月    面積/築年数比  エリア人気度スコア  年別価格トレンド  価格増加率   駅距離_面積比  \
0  202006  ...  31.0    6   2.419355        0.0       0.0    0.0  0.346667   
1  202003  ...   4.0    3  13.750000        0.0       0.0    0.0  0.018182   
2  202006  ...  31.0    6   0.483871        0.0       0.0    0.0  0.133333   
3  202006  ...  29.0    6   1.551724        0.0       0.0    0.0  0.044444   
4  202006  ...  39.0    6   0.512821        0.0       0.0    0.0  0.150000   

   容積率_建ぺい率比   面積×容積率  建築年×建ぺい率  
0        1.5   4500.0   7956